In [89]:
import numpy as np
import pandas as pd
import datetime
import math

In [90]:
dam = pd.read_csv('dam.csv')
dam

,hourlyid,market,tradedate,hour,price,volume
0,26345,1,2016-01-01,0,40.10,477.5
1,26346,1,2016-01-01,1,40.08,486.6
2,26347,1,2016-01-01,2,40.06,495.6
3,26348,1,2016-01-01,3,40.08,496.6
4,26349,1,2016-01-01,4,40.07,502.9
...,...,...,...,...,...,...
46723,73029,1,2021-04-29,4,39.97,126.1
46724,73037,1,2021-04-29,12,68.06,71.0
46725,73049,1,2021-04-30,0,44.04,59.6
46726,73057,1,2021-04-30,8,190.00,72.0


In [93]:
dam['date']=pd.to_datetime(dam['tradedate'],format="%Y/%m/%d")#"%d/%m/%Y")
dam=dam.set_index(pd.DatetimeIndex(dam['date']))
dam['year']=dam.index.year
dam['month']=dam.index.month
dam['day']=dam.index.day
dam['day_of_week']=dam.index.dayofweek
dam['month_no']=dam['year']+dam['month']/100

dam['hour_ref'] = dam.apply(lambda x: ('weekend_' + str(x['hour']) if x['day_of_week']>4 else 'weekday_' + str(x['hour'])),axis=1)
dam

,hourlyid,market,tradedate,hour,price,volume,date,year,month,day,day_of_week,month_no,hour_ref
date,,,,,,,,,,,,,
2016-01-01,26345,1,2016-01-01,0,40.10,477.5,2016-01-01,2016,1,1,4,2016.01,weekday_0
2016-01-01,26346,1,2016-01-01,1,40.08,486.6,2016-01-01,2016,1,1,4,2016.01,weekday_1
2016-01-01,26347,1,2016-01-01,2,40.06,495.6,2016-01-01,2016,1,1,4,2016.01,weekday_2
2016-01-01,26348,1,2016-01-01,3,40.08,496.6,2016-01-01,2016,1,1,4,2016.01,weekday_3
2016-01-01,26349,1,2016-01-01,4,40.07,502.9,2016-01-01,2016,1,1,4,2016.01,weekday_4
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-04-29,73029,1,2021-04-29,4,39.97,126.1,2021-04-29,2021,4,29,3,2021.04,weekday_4
2021-04-29,73037,1,2021-04-29,12,68.06,71.0,2021-04-29,2021,4,29,3,2021.04,weekday_12
2021-04-30,73049,1,2021-04-30,0,44.04,59.6,2021-04-30,2021,4,30,4,2021.04,weekday_0


In [94]:
monthly=dam.groupby(['month_no','year','month','hour', 'hour_ref']).mean().reset_index(['hour','month','year'])
monthly=monthly.drop(['day'],1)
monthly.rename(columns={'price': 'price_mean'}, inplace=True)
monthly['volume_sum']=dam[['month_no','hour_ref','volume']].groupby(['month_no','hour_ref']).sum()
monthly['price_stdv']=dam[['month_no','hour_ref','price']].groupby(['month_no','hour_ref']).std()
monthly['price_max']=dam[['month_no','hour_ref','price']].groupby(['month_no','hour_ref']).max()
monthly['price_min']=dam[['month_no','hour_ref','price']].groupby(['month_no','hour_ref']).min()
monthly['price_coefvar']=monthly['price_stdv']/monthly['price_mean']*100
monthly['price_range']=monthly['price_max'] - monthly['price_min']
monthly['hours']=dam[['month_no','hour_ref']].groupby(['month_no','hour_ref']).size()
monthly['power']=monthly['volume_sum']/monthly['hours']
monthly

year  month  hour      hourlyid  market  price_mean  \
month_no hour_ref                                                          
2016.01  weekday_0   2016      1     0  26699.285714     1.0   48.505238   
         weekend_0   2016      1     0  26717.000000     1.0   48.618000   
         weekday_1   2016      1     1  26700.285714     1.0   48.434286   
         weekend_1   2016      1     1  26718.000000     1.0   48.459000   
         weekday_2   2016      1     2  26701.285714     1.0   48.385714   
...                   ...    ...   ...           ...     ...         ...   
2021.04  weekend_21  2021      4    21  72686.000000     1.0   38.762500   
         weekday_22  2021      4    22  72736.090909     1.0   46.260455   
         weekend_22  2021      4    22  72687.000000     1.0   40.628750   
         weekday_23  2021      4    23  72737.090909     1.0   41.821818   
         weekend_23  2021      4    23  72688.000000     1.0   35.863750   

                         volume  day_of_week  volume_sum  price_stdv  \
month_no hour_ref                                                      
2016.01  weekday_0   332.371429     2.095238      6979.8    3.363676   
         weekend_0   372.530000     5.500000      3725.3    2.717061   
         weekday_1   336.747619     2.095238      7071.7    3.315599   
         weekend_1   372.900000     5.500000      3729.0    2.739337   
         weekday_2   339.400000     2.095238      7127.4    3.287444   
...                         ...          ...         ...         ...   
2021.04  weekend_21  105.000000     5.500000       840.0    2.319875   
         weekday_22   94.922727     2.136364      2088.3    5.526819   
         weekend_22  118.825000     5.500000       950.6    4.150692   
         weekday_23   96.250000     2.136364      2117.5    3.883199   
         weekend_23  160.075000     5.500000      1280.6    5.101591   

                     price_max  price_min  price_coefvar  price_range  hours  \
month_no hour_ref                                                              
2016.01  weekday_0       52.02      40.10       6.934665        11.92     21   
         weekend_0       52.06      43.95       5.588590         8.11     10   
         weekday_1       51.65      40.08       6.845562        11.57     21   
         weekend_1       52.06      43.80       5.652896         8.26     10   
         weekday_2       51.65      40.06       6.794244        11.59     21   
...                        ...        ...            ...          ...    ...   
2021.04  weekend_21      40.09      34.96       5.984844         5.13      8   
         weekday_22      55.80      35.05      11.947178        20.75     22   
         weekend_22      45.04      34.96      10.216144        10.08      8   
         weekday_23      45.08      35.00       9.285103        10.08     22   
         weekend_23      40.08      27.10      14.224922        12.98      8   

                          power  
month_no hour_ref                
2016.01  weekday_0   332.371429  
         weekend_0   372.530000  
         weekday_1   336.747619  
         weekend_1   372.900000  
         weekday_2   339.400000  
...                         ...  
2021.04  weekend_21  105.000000  
         weekday_22   94.922727  
         weekend_22  118.825000  
         weekday_23   96.250000  
         weekend_23  160.075000  

[3072 rows x 16 columns]

In [134]:
dam_filt=dam[(dam.year<2020)]
cal_monthly=dam_filt[['month','hour_ref','price']].groupby(['month','hour_ref']).mean()

cal_monthly.rename(columns={'price': 'price_mean'}, inplace=True)
cal_monthly['hours']=dam_filt[['month','hour_ref']].groupby(['month', 'hour_ref']).size()
cal_monthly['volume_sum']=dam_filt[['month','hour_ref','volume']].groupby(['month','hour_ref']).sum()
cal_monthly['power']=cal_monthly['volume_sum']/cal_monthly['hours']
cal_monthly['volume_mean']=cal_monthly['volume_sum']/4
cal_monthly=cal_monthly.drop(['volume_sum'],1)
cal_monthly

price_mean  hours       power  volume_mean
month hour_ref                                              
1     weekday_0    33.593483     89  263.117978     5854.375
      weekday_1    33.247303     89  262.165169     5833.175
      weekday_10   74.164831     89   86.859551     1932.625
      weekday_11   72.023371     89   90.902247     2022.575
      weekday_12   71.558539     89   93.005618     2069.375
...                      ...    ...         ...          ...
12    weekend_5    32.883158     38  257.263158     2444.000
      weekend_6    34.064737     38  241.797368     2297.075
      weekend_7    46.835526     38  216.147368     2053.400
      weekend_8    52.244211     38  207.623684     1972.425
      weekend_9    53.827105     38  206.878947     1965.350

[576 rows x 4 columns]

In [135]:
monthly.to_csv('dam_monthly_hours.csv')

In [136]:
cal_monthly.to_csv('dam_cal_monthly_hours.csv')

In [130]:
dam.to_csv('dam_out.csv')

In [107]:
months=dam['month_no'].unique().tolist()
months
for month in months:
    days=dam.loc[dam.month_no==month]
    days.to_csv('months/dam_hourly_' + str(month) + '.csv')

In [114]:
dam['price_vol']=dam['price']*dam['volume']
daily=dam.groupby(['year','month','day']).sum().drop(['hour','day_of_week','month_no','market','hourlyid'],axis=1)
daily['price_weight_av']=daily['price_vol']/daily['volume']
daily=daily.drop(['price','price_vol'], axis=1)

daily

volume  price_weight_av
year month day                         
2016 1     1    7873.8        42.785251
           2    7319.5        47.268532
           3    7198.7        48.857812
           4    4457.0        50.693024
           5    3750.4        47.746609
...                ...              ...
2021 4     26   2092.9        88.683281
           27   2048.2        82.150912
           28   2130.2        83.346161
           29   2134.0        83.738512
           30   2198.9        85.068659

[1947 rows x 2 columns]

In [125]:
daily['price_max']=dam[['year','month','day','price']].groupby(['year','month','day']).max()
daily['price_min']=dam[['year','month','day','price']].groupby(['year','month','day']).min()

In [126]:
daily.to_csv('dam_daily.csv')